https://docs.ray.io/en/latest/rllib/rllib-training.html

Example: Querying a policy’s action distribution



In [1]:
import ray

In [2]:
import tensorflow

In [3]:
import torch

In [4]:
import numpy as np
# Get a reference to the policy
from ray.rllib.agents.ppo import PPOTrainer
trainer = PPOTrainer(env="CartPole-v0", config={"framework": "tf2", "num_workers": 0})
policy = trainer.get_policy()

Metal device set to: Apple M1


Install gputil for GPU system monitoring.


In [15]:
# Run a forward pass to get model output logits. Note that complex observations
# must be preprocessed as in the above code block.
logits, _ = policy.model({"obs": np.array([[0.1, 0.2, 0.3, 0.4]])})

In [16]:
# version in docs
# logits, _ = policy.model.from_batch({"obs": np.array([[0.1, 0.2, 0.3, 0.4]])})

In [6]:
policy.dist_class

ray.rllib.models.tf.tf_action_dist.Categorical

In [7]:
dist = policy.dist_class(logits, policy.model)

In [8]:
dist.sample()

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [9]:
dist.logp([1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.69162846], dtype=float32)>

In [10]:
policy.model.value_function()

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0009751], dtype=float32)>

In [11]:
policy.model.base_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 observations (InputLayer)      [(None, 4)]          0           []                               
                                                                                                  
 fc_1 (Dense)                   (None, 256)          1280        ['observations[0][0]']           
                                                                                                  
 fc_value_1 (Dense)             (None, 256)          1280        ['observations[0][0]']           
                                                                                                  
 fc_2 (Dense)                   (None, 256)          65792       ['fc_1[0][0]']                   
                                                                                              